## インストール

In [ ]:
# pip install pingouin
# pip install plotnine
# pip install mysql-connector-python
# pip install numexpr
# pip install python-dotenv
# pip install japanize-matplotlib




In [ ]:
pip install xlwt


In [ ]:
from itertools import combinations

import csv
import MySQLdb
import mysql.connector as mydb
import pandas as pd
import pandas.io.sql as psql
import numpy as np
from pingouin import kruskal, mwu, multicomp
from plotnine import *


In [ ]:
# %load_ext dotenv
# %dotenv

%reload_ext dotenv


In [ ]:
host = os.environ.get('HOST')
user = os.environ.get('USER_NAME')
password = os.environ.get('PASSWORD')
database = os.environ.get('DATABASE')
question_database = os.environ.get('QUESTION_UI_DATABASE')
DB_CONFIG = dict(host=host, user=user, password=password, database=database)
QUESTION_DB_CONFIG = dict(
    host=host, user=user, password=password, database=question_database)


def check_db_connection(db_config=QUESTION_DB_CONFIG):
  connection = mydb.connect(**db_config)
  connection.ping(reconnect=True)
  # 接続できているかどうか確認
  print(connection.is_connected())


check_db_connection(QUESTION_DB_CONFIG)


In [ ]:
UI_CONDITION = ("question", "perspective", "non")

THEME_CONDITION = ("tpp", "airPollution", "vietnamTrip")

TPP_UI_CONDITION = (
    "TPP_WITH_QUERYUI",
    "TPP_WITH_NONUI",
)
AIRPOLLUTION_UI_CONDITION = (
    "AIRPOLLUTION_WITH_QUERYUI",
    "AIRPOLLUTION_WITH_NONUI",
)
VIETNAMTRIP_UI_CONDITION = (
    "VIETNAMTRIP_WITH_QUERYUI",
    "VIETNAMTRIP_WITH_NONUI"
)


In [ ]:

def get_dataframe_from_db(sql, db_config):
  """
  db上のデータをデータフレームとして取得
  """
  conn = mydb.connect(**db_config)
  df_read = pd.read_sql(sql, conn)
  conn.close()
  return df_read


def convert_csv_to_array(csv_path):
  """
  csvファイルの中身を一次元配列へ変換
  """
  result_arr = []
  with open(csv_path) as f:
      reader = csv.reader(f)
      for row in reader:
        result_arr.append(str(row[0]))
  return result_arr


def convert_singleArray_to_dataframe(array, columnName):
  df = pd.DataFrame(array, columns=[columnName],)
  return df


In [ ]:
users = get_dataframe_from_db('select * from users;', DB_CONFIG)
res = users.query('taskId == "AIRPOLLUTION_WITH_QUERYUI"').crowdId.tolist()
print(res)


## 分析対象とする実験協力者

- ウェブ検索タスクにて意見文章の提出まで行い，事後アンケートへの回答が完了しているワーカー
- タスク所要時間が外れ値ではないワーカー

In [ ]:

# ウェブ検索タスクと事後アンケートへの回答の両方が完了しているワーカー
questionnaire_answered_crowdIds_csv_path = './data/questionnaire_answered_crowdId.csv'
questionnaire_answered_crowdIds = convert_csv_to_array(questionnaire_answered_crowdIds_csv_path)
questionnaire_answered_crowdIds_df = convert_singleArray_to_dataframe(
    questionnaire_answered_crowdIds, 'crowdId')
users_who_complete_webSearchTask_df = get_dataframe_from_db("select * from users where hasFinishedWebSearchTask", DB_CONFIG)
users_who_completed_all_task_df =  users_who_complete_webSearchTask_df[users_who_complete_webSearchTask_df['crowdId'].isin(questionnaire_answered_crowdIds)]

all_task_done_user_crowdIds = users_who_completed_all_task_df['crowdId'].tolist()

# タスク所要時間の外れ値処理
def get_dataframe_without_outlier(df, target_column):
    q1 = df[target_column].quantile(
        0.25)
    q3 = df[target_column].quantile(
        0.75)
    iqr = q3 - q1
    threshold_upper = q3 + 1.5 * iqr
    threshold_lower = q1 - 1.5 * iqr
    df["threshold_lower"] = threshold_lower
    df["threshold_upper"] = threshold_upper

    df_without_outlier = df[(df[target_column] < threshold_upper) & (
        df[target_column] > threshold_lower)]
    return df_without_outlier

task_time_df = get_dataframe_from_db("select * from taskTime;", DB_CONFIG)
task_time_with_users_done_all_task_df = task_time_df[task_time_df['crowdId'].isin(
    all_task_done_user_crowdIds)]
task_time_with_users_done_all_task_df["taskTimeDuration"] = (
    task_time_with_users_done_all_task_df["finished_at"]-task_time_with_users_done_all_task_df["reStarted_at"]).map(lambda x: x.total_seconds())

valid_users_df = get_dataframe_without_outlier(
    task_time_with_users_done_all_task_df, 'taskTimeDuration')
valid_users_df.head()
valid_user_crowdIds = valid_users_df.crowdId.tolist()
print(len(valid_user_crowdIds))
# print(len(valid_users_df)) # 264人

## 問いかけ群ワーカー
# ウェブ検索タスクと事後アンケートへの回答の両方が完了している問いかけ群ワーカー
questionnaire_answered_questionUI_crowdIds_csv_path = './data/questionnaire_answered_questionUI_crowdId.csv'
questionnaire_answered_questionUI_crowdIds = convert_csv_to_array(
    questionnaire_answered_questionUI_crowdIds_csv_path)
questionnaire_answered_questionUI_crowdIds_df = convert_singleArray_to_dataframe(
    questionnaire_answered_questionUI_crowdIds, 'crowdId')
q_users_df = get_dataframe_from_db(
    "select * from users where hasFinishedWebSearchTask;", QUESTION_DB_CONFIG)
q_users_who_completed_all_task_df = q_users_df[q_users_df['crowdId'].isin(
    questionnaire_answered_questionUI_crowdIds)]
q_all_task_done_user_crowdIds = q_users_who_completed_all_task_df['crowdId'].tolist()

q_task_time_df = get_dataframe_from_db('select * from taskTime;', QUESTION_DB_CONFIG)
q_task_time_with_users_done_all_task_df = q_task_time_df[q_task_time_df['crowdId'].isin(
    q_all_task_done_user_crowdIds)]
q_task_time_with_users_done_all_task_df["taskTimeDuration"] = (
    q_task_time_with_users_done_all_task_df["finished_at"]-q_task_time_with_users_done_all_task_df["reStarted_at"]).map(lambda x: x.total_seconds())
q_valid_users_df = get_dataframe_without_outlier(
    q_task_time_with_users_done_all_task_df, 'taskTimeDuration')
q_valid_user_crowdIds = q_valid_users_df.crowdId.tolist()
print(len(q_valid_user_crowdIds)) # 87人




## 実験協力者の内訳

- questionUI群に関しては，別データで分析を行うため，perspectiveUI，nonUIのみ集計する

In [ ]:
THEME_UI_CONDITION = (
                        "TPP_WITH_QUERYUI",
                        "AIRPOLLUTION_WITH_QUERYUI",
                        "VIETNAMTRIP_WITH_QUERYUI",
                        "TPP_WITH_NONUI",
                        "AIRPOLLUTION_WITH_NONUI",
                        "VIETNAMTRIP_WITH_NONUI"
                        )

users_df = get_dataframe_from_db('select * from users;', DB_CONFIG)
users_df = users_df[users_df['crowdId'].isin(
    valid_user_crowdIds)]

for theme_ui_condition in THEME_UI_CONDITION:
    res = users_df.query('taskId == @theme_ui_condition')

# 問いかけ群ワーカー
QUESTION_UI_CONDITION = (
    "TPP_WITH_QUESTIONUI",
    "AIRPOLLUTION_WITH_QUESTIONUI",
    "VIETNAMTRIP_WITH_QUESTIONUI"
)

q_users_df = get_dataframe_from_db("select * from users;", QUESTION_DB_CONFIG)
q_users_df = q_users_df[q_users_df['crowdId'].isin(q_valid_user_crowdIds)]



for condition in QUESTION_UI_CONDITION:
    res = q_users_df.query('taskId == @condition')
    print(len(res))

question_UI_participant = [27, 32, 28]
perspective_UI_participant = [32, 30, 25]
non_UI_participant = [33, 35, 30]

participant_list = [
    question_UI_participant,
    perspective_UI_participant,
    non_UI_participant
]

participant_df = pd.DataFrame(participant_list)
participant_df.index = ['questionUI','perspectiveUI', 'nonUI']
participant_df.columns = ['TPP', '大気汚染', 'ベトナム旅行']
participant_df.head()





In [ ]:

# valid_user_crowdIds
# q_valid_user_crowdIds

def get_dataframe_without_invalid_user(df, valid_crowdIds):
  return df[df['crowdId'].isin(valid_crowdIds)]


## 意見文章の抽出

In [ ]:
q_users_who_completed_all_task_df

# 大気汚染またはベトナム旅行がテーマだったワーカー
q_users_with_airPollution_or_vietnam_df = q_users_df[q_users_df['taskId'].isin(['AIRPOLLUTION_WITH_QUESTIONUI','VIETNAMTRIP_WITH_QUESTIONUI'])]
# 大気汚染またはベトナム旅行がテーマだったワーカーのcrowdId
q_crowdIds_with_airPollution_or_vietnam = q_users_with_airPollution_or_vietnam_df.crowdId.tolist()


q_opinion_document_df = get_dataframe_from_db('select * from opinionDocuments;', QUESTION_DB_CONFIG)
q_opinion_document_without_invalid_users = get_dataframe_without_invalid_user(
    q_opinion_document_df, q_crowdIds_with_airPollution_or_vietnam)
q_opinion_document_without_invalid_users.head()
q_opinion_document_without_invalid_users.to_csv('data/test.csv')
# q_opinion_document_list = q_opinion_document_without_invalid_users.content.tolist()

## 評価用シートの作成（csv化）
# csv_path = 'opinion.csv'

# with open(csv_path, "w", encoding="utf_8_sig") as f:
#     writer = csv.writer(f, lineterminator='\n')
#     for i in q_opinion_document_list:
#         writer.writerow([i])







## 集計結果

- 中央値
- 標準偏差

In [ ]:

AIRPOLLUTION_UI_CONDITION = (
    "AIRPOLLUTION_WITH_QUERYUI",
    "AIRPOLLUTION_WITH_NONUI",
)

VIETNAMTRIP_UI_CONDITION = (
    "VIETNAMTRIP_WITH_QUERYUI",
    "VIETNAMTRIP_WITH_NONUI"
)

QUESTION_UI_CONDITION = (
    "AIRPOLLUTION_WITH_QUESTIONUI",
    "VIETNAMTRIP_WITH_QUESTIONUI"
)

def get_target_column_median_and_std(df, target_column, condition_list):
    for condition in condition_list:
        target_df = df.query('taskId == @condition')
        median_value = target_df[target_column].median()
        std_value = target_df[target_column].std()
        print('=======')
        print(condition)
        print('中央値', median_value, '標準偏差', std_value)

# タスク所要時間
task_time_sql = """
select
	tt.crowdId, tt.restarted_at, tt.finished_at, users.taskId
from
	taskTime as tt
inner join
	users on tt.crowdId = users.crowdId
"""
# 問いかけ群以外
task_time_df = get_dataframe_from_db(task_time_sql, DB_CONFIG)
task_time_df_without_invalid_user = get_dataframe_without_invalid_user(
    task_time_df, valid_user_crowdIds)
task_time_df_without_invalid_user["taskTimeDuration"] = (
    task_time_df_without_invalid_user["finished_at"]-task_time_df_without_invalid_user["restarted_at"]).map(lambda x: x.total_seconds())
# 問いかけ群
q_task_time_df = get_dataframe_from_db(task_time_sql, QUESTION_DB_CONFIG)
q_task_time_df_without_invalid_user = get_dataframe_without_invalid_user(
    q_task_time_df, q_valid_user_crowdIds)
q_task_time_df_without_invalid_user["taskTimeDuration"] = (
    q_task_time_df_without_invalid_user["finished_at"]-q_task_time_df_without_invalid_user["restarted_at"]).map(lambda x: x.total_seconds())

# 大気汚染 && クエリ所要時間 && 問いかけ群以外
get_target_column_median_and_std(
    task_time_df_without_invalid_user, 'taskTimeDuration', AIRPOLLUTION_UI_CONDITION)
# ベトナム旅行 && クエリ所要時間 && 問いかけ群以外
get_target_column_median_and_std(
    task_time_df_without_invalid_user, 'taskTimeDuration', VIETNAMTRIP_UI_CONDITION)
# タスク所要時間 && 問いかけ群
get_target_column_median_and_std(
    q_task_time_df_without_invalid_user, 'taskTimeDuration', QUESTION_UI_CONDITION)


In [ ]:
# クエリ発行回数
query_count_sql = """
select
    users.crowdId, users.taskId, count(users.crowdId) as query_count
from
    queryLogs as ql
inner join
    users on ql.crowdId = users.crowdId
group by
    users.crowdId, users.taskId
"""

# 問いかけ群以外
query_count_df = get_dataframe_from_db(query_count_sql, DB_CONFIG)
query_count_df_without_invalid_user = get_dataframe_without_invalid_user(
    query_count_df, valid_user_crowdIds)

# 問いかけ群
q_query_count_df = get_dataframe_from_db(query_count_sql, QUESTION_DB_CONFIG)
q_query_count_df_without_invalid_user = get_dataframe_without_invalid_user(
    q_query_count_df, q_valid_user_crowdIds)

# 大気汚染 && クエリ発行回数 && 問いかけ群以外
get_target_column_median_and_std(
    query_count_df_without_invalid_user, 'query_count', AIRPOLLUTION_UI_CONDITION)
# 大気汚染 && クエリ発行回数 && 問いかけ群以外
get_target_column_median_and_std(
    query_count_df_without_invalid_user, 'query_count', VIETNAMTRIP_UI_CONDITION)
# クエリ発行回数 && 問いかけ群
get_target_column_median_and_std(
    q_query_count_df_without_invalid_user, 'query_count', QUESTION_UI_CONDITION)


In [ ]:

# 詳細ページ閲覧回数
visit_web_page_count_sql = """
select
	users.crowdId, users.taskId, count(users.crowdId) as visit_web_page_count
from
	visitPageLogs vl
inner join
	users on vl.crowdId = users.crowdId
group by
  users.crowdId, users.taskId;
"""

# 問いかけ群以外
visit_web_page_count_df = get_dataframe_from_db(visit_web_page_count_sql, DB_CONFIG)
visit_web_page_count_df_without_invalid_user = get_dataframe_without_invalid_user(
    visit_web_page_count_df, valid_user_crowdIds)
# 問いかけ群
q_visit_web_page_count_df = get_dataframe_from_db(
    visit_web_page_count_sql, QUESTION_DB_CONFIG)
q_visit_web_page_count_df_without_invalid_user = get_dataframe_without_invalid_user(
    q_visit_web_page_count_df, q_valid_user_crowdIds)

# 大気汚染 && 詳細ページ閲覧時間 && 問いかけ群以外
get_target_column_median_and_std(
    visit_web_page_count_df_without_invalid_user, 'visit_web_page_count', AIRPOLLUTION_UI_CONDITION)
# 大気汚染 && 詳細ページ閲覧時間 && 問いかけ群以外
get_target_column_median_and_std(
    visit_web_page_count_df_without_invalid_user, 'visit_web_page_count', VIETNAMTRIP_UI_CONDITION)
# 詳細ページ閲覧時間 && 問いかけ群
get_target_column_median_and_std(
    q_visit_web_page_count_df_without_invalid_user, 'visit_web_page_count', QUESTION_UI_CONDITION)

In [ ]:
# SERP閲覧時間
serp_time_duration_sql = """
select
	users.crowdId, users.taskId, st.time_on_serp
from
	serpTimeDuration as st
inner join
	users on st.crowdId = users.crowdId;
"""

# 問いかけ群以外
serp_time_duration_df = get_dataframe_from_db(
    serp_time_duration_sql, DB_CONFIG)
serp_time_duration_df_without_invalid_user = get_dataframe_without_invalid_user(
    serp_time_duration_df, valid_user_crowdIds)
# 問いかけ群
q_serp_time_duration_df = get_dataframe_from_db(
    serp_time_duration_sql, QUESTION_DB_CONFIG)
q_serp_time_duration_df_without_invalid_user = get_dataframe_without_invalid_user(
    q_serp_time_duration_df , q_valid_user_crowdIds)

# 大気汚染 && 詳細ページ閲覧時間 && 問いかけ群以外
get_target_column_median_and_std(
    serp_time_duration_df_without_invalid_user, 'time_on_serp', AIRPOLLUTION_UI_CONDITION)
# 大気汚染 && 詳細ページ閲覧時間 && 問いかけ群以外
get_target_column_median_and_std(
    serp_time_duration_df_without_invalid_user, 'time_on_serp', VIETNAMTRIP_UI_CONDITION)
# 詳細ページ閲覧時間 && 問いかけ群
get_target_column_median_and_std(
    q_serp_time_duration_df_without_invalid_user, 'time_on_serp', QUESTION_UI_CONDITION)

In [ ]:
# 最大クリック深度
max_ranking_sql = """
select
	vp.crowdId, max(ranking) as max_ranking, users.taskId
from
	visitPageLogs vp
inner join
	users on users.crowdId = vp.crowdId
group by
	crowdId, users.taskId;
"""

# 問いかけ群以外
max_ranking_df = get_dataframe_from_db(
    max_ranking_sql, DB_CONFIG)
max_ranking_df_without_invalid_user = get_dataframe_without_invalid_user(
    max_ranking_df, valid_user_crowdIds)
# 問いかけ群
q_max_ranking_df = get_dataframe_from_db(
    max_ranking_sql, QUESTION_DB_CONFIG)
q_max_ranking_df_without_invalid_user = get_dataframe_without_invalid_user(
    q_max_ranking_df, q_valid_user_crowdIds)

# 大気汚染 && 詳細ページ閲覧時間 && 問いかけ群以外
get_target_column_median_and_std(
    max_ranking_df_without_invalid_user, 'max_ranking', AIRPOLLUTION_UI_CONDITION)
# 大気汚染 && 詳細ページ閲覧時間 && 問いかけ群以外
get_target_column_median_and_std(
    max_ranking_df_without_invalid_user, 'max_ranking', VIETNAMTRIP_UI_CONDITION)
# 詳細ページ閲覧時間 && 問いかけ群
get_target_column_median_and_std(
    q_max_ranking_df_without_invalid_user, 'max_ranking', QUESTION_UI_CONDITION)



## 行動指標の分析
- クエリ発行回数
- タスク所要時間
- 詳細ページ閲覧時間
- SERP滞在時間
- 最大クリック深度

各トピックごとに，1要因（UI）で分散分析を行う．
水準は3つ
- questionUI
- perspectiveUI
- nonUI


In [ ]:
AIRPOLLUTION_CONDITION = (
    "AIRPOLLUTION_WITH_QUESTIONUI",
    "AIRPOLLUTION_WITH_QUERYUI",
    "AIRPOLLUTION_WITH_NONUI",
)

VIETNAMTRIP_CONDITION = (
    "VIETNAMTRIP_WITH_QUESTIONUI",
    "VIETNAMTRIP_WITH_QUERYUI",
    "VIETNAMTRIP_WITH_NONUI"
)

# 多重比較
def do_multiple_comparison_with_FDR_airPollution(df, target_value, target_task=None):
    # 多重比較 with Benjamini-Hochberg法
    pvals = {}
    for ui_1, ui_2 in combinations(AIRPOLLUTION_CONDITION, 2):
        if target_task:
            x = df[(df.task_type == target_task) & (
                df.taskId == ui_1)][target_value]
            y = df[(df.task_type == target_task) & (
                df.taskId == ui_2)][target_value]
        else:
            x = df[df.taskId == ui_1][target_value]
            y = df[df.taskId == ui_2][target_value]
        pval = float(mwu(x, y)['p-val'])
        pvals[(ui_1, ui_2)] = pval
    else:
        rejects, pvals_corrs = multicomp(list(pvals.values()), method='fdr_bh')
        result = {}
        for ui_pair, reject, pval_corr in zip(list(pvals.keys()), rejects, pvals_corrs):
            result[ui_pair] = (reject, pval_corr)
        return result


def do_multiple_comparison_with_FDR_vietnam(df, target_value, target_task=None):
    # 多重比較 with Benjamini-Hochberg法
    pvals = {}
    for ui_1, ui_2 in combinations(VIETNAMTRIP_CONDITION, 2):
        if target_task:
            x = df[(df.task_type == target_task) & (
                df.taskId == ui_1)][target_value]
            y = df[(df.task_type == target_task) & (
                df.taskId == ui_2)][target_value]
        else:
            x = df[df.taskId == ui_1][target_value]
            y = df[df.taskId == ui_2][target_value]
        pval = float(mwu(x, y)['p-val'])
        pvals[(ui_1, ui_2)] = pval
    else:
        rejects, pvals_corrs = multicomp(list(pvals.values()), method='fdr_bh')
        result = {}
        for ui_pair, reject, pval_corr in zip(list(pvals.keys()), rejects, pvals_corrs):
            result[ui_pair] = (reject, pval_corr)
        return result


In [ ]:

# タスク所要時間
task_time_sql = """
select
	tt.crowdId, tt.restarted_at, tt.finished_at, users.taskId
from
	taskTime as tt
inner join
	users on tt.crowdId = users.crowdId
"""

task_time_df = get_dataframe_from_db(task_time_sql, DB_CONFIG)
task_time_df_without_invalid_user = get_dataframe_without_invalid_user(
    task_time_df, valid_user_crowdIds)
task_time_df_without_invalid_user["taskTimeDuration"] = (
    task_time_df_without_invalid_user["finished_at"]-task_time_df_without_invalid_user["restarted_at"]
    ).map(lambda x: x.total_seconds())


q_task_time_df = get_dataframe_from_db(task_time_sql, QUESTION_DB_CONFIG)
q_task_time_df_without_invalid_user = get_dataframe_without_invalid_user(
    q_task_time_df, q_valid_user_crowdIds)
q_task_time_df_without_invalid_user["taskTimeDuration"] = (
    q_task_time_df_without_invalid_user["finished_at"]-q_task_time_df_without_invalid_user["restarted_at"]
    ).map(lambda x: x.total_seconds())

# 大気汚染
airPollution_task_time_df = task_time_df_without_invalid_user[
    task_time_df_without_invalid_user['taskId'].isin(AIRPOLLUTION_UI_CONDITION)
]
q_airPollution_task_time_df = q_task_time_df_without_invalid_user[
    q_task_time_df_without_invalid_user['taskId'].isin(['AIRPOLLUTION_WITH_QUESTIONUI'])
]

all_condition_airPollution_task_time_df = pd.concat([
    airPollution_task_time_df,
    q_airPollution_task_time_df
])

anova_result = kruskal(data=all_condition_airPollution_task_time_df,dv="taskTimeDuration", between='taskId')
display(anova_result)
if int(anova_result['p-unc'] < 0.05):
    mcomp_result = do_multiple_comparison_with_FDR_airPollution(
        all_condition_airPollution_task_time_df, target_value='query_count')
    display(mcomp_result)

# ベトナム旅行
vietnam_task_time_df = task_time_df_without_invalid_user[task_time_df_without_invalid_user['taskId'].isin(
    VIETNAMTRIP_UI_CONDITION)]
q_vietnam_task_time_df = q_task_time_df_without_invalid_user[q_task_time_df_without_invalid_user['taskId'].isin(['VIETNAMTRIP_WITH_QUESTIONUI'])]
all_condition_vietnam_task_time_df = pd.concat(
    [vietnam_task_time_df, q_vietnam_task_time_df])
anova_result = kruskal(
    data=all_condition_vietnam_task_time_df,
    dv="taskTimeDuration",
    between='taskId'
)
display(anova_result)
if int(anova_result['p-unc'] < 0.05):
    mcomp_result = do_multiple_comparison_with_FDR_vietnam(
        all_condition_airPollution_task_time_df, target_value='query_count')
    display(mcomp_result)


In [ ]:
# クエリ発行回数
query_count_sql = """
select
    users.crowdId, users.taskId, count(users.crowdId) as query_count
from
    queryLogs as ql
inner join
    users on ql.crowdId = users.crowdId
group by
    users.crowdId, users.taskId
"""

# 問いかけ群以外
query_count_df = get_dataframe_from_db(query_count_sql, DB_CONFIG)
query_count_df_without_invalid_user = get_dataframe_without_invalid_user(
    query_count_df, valid_user_crowdIds)

# 問いかけ群
q_query_count_df = get_dataframe_from_db(query_count_sql, QUESTION_DB_CONFIG)
q_query_count_df_without_invalid_user = get_dataframe_without_invalid_user(
    q_query_count_df, q_valid_user_crowdIds)

# 大気汚染
airPollution_query_count_df = query_count_df_without_invalid_user[
    query_count_df_without_invalid_user['taskId'].isin(AIRPOLLUTION_UI_CONDITION)
]
q_airPollution_query_count_df = q_query_count_df_without_invalid_user[
    q_query_count_df_without_invalid_user['taskId'].isin(['AIRPOLLUTION_WITH_QUESTIONUI'])
]
all_condition_airPollution_query_count_df = pd.concat(
    [airPollution_query_count_df, q_airPollution_query_count_df])
anova_result = kruskal(
    data=all_condition_airPollution_query_count_df,
    dv="query_count",
    between='taskId'
)
display(anova_result)
if int(anova_result['p-unc'] < 0.05):
    mcomp_result = do_multiple_comparison_with_FDR_airPollution(
        all_condition_airPollution_query_count_df, target_value='query_count')
    display(mcomp_result)

# ベトナム旅行
vietnam_query_count_df = query_count_df_without_invalid_user[
    query_count_df_without_invalid_user['taskId'].isin(VIETNAMTRIP_UI_CONDITION)
]
q_vietnam_query_count_df = q_query_count_df_without_invalid_user[
    q_query_count_df_without_invalid_user['taskId'].isin(['VIETNAMTRIP_WITH_QUESTIONUI'])
]
all_condition_vietnam_query_count_df = pd.concat([
    vietnam_query_count_df,
    q_vietnam_query_count_df
])
anova_result = kruskal(data=all_condition_vietnam_query_count_df, dv="query_count", between='taskId')
display(anova_result)
if int(anova_result['p-unc'] < 0.05):
    mcomp_result = do_multiple_comparison_with_FDR_vietnam(
    all_condition_vietnam_query_count_df, target_value='query_count')
    display(mcomp_result)


### 詳細ページ閲覧回数

In [ ]:
# 大気汚染
airPollution_visit_web_page_count_df = visit_web_page_count_df_without_invalid_user[
    visit_web_page_count_df_without_invalid_user['taskId'].isin(
        AIRPOLLUTION_UI_CONDITION)
]
q_airPollution_visit_web_page_count_df = q_visit_web_page_count_df_without_invalid_user[
    q_visit_web_page_count_df_without_invalid_user['taskId'].isin(
        ['AIRPOLLUTION_WITH_QUESTIONUI'])
]
all_condition_airPollution_visit_web_page_count_df = pd.concat(
    [airPollution_visit_web_page_count_df, q_airPollution_visit_web_page_count_df])
anova_result = kruskal(
    data=all_condition_airPollution_visit_web_page_count_df,
    dv="visit_web_page_count",
    between='taskId'
)
display(anova_result)
if int(anova_result['p-unc'] < 0.05):
    mcomp_result = do_multiple_comparison_with_FDR_airPollution(
        all_condition_airPollution_visit_web_page_count_df, target_value='visit_web_page_count')
    display(mcomp_result)

# ベトナム旅行
vietnam_visit_web_page_count_df = visit_web_page_count_df_without_invalid_user[
    visit_web_page_count_df_without_invalid_user['taskId'].isin(
        VIETNAMTRIP_UI_CONDITION)
]
q_vietnam_visit_web_page_count_df = q_visit_web_page_count_df_without_invalid_user[
    q_visit_web_page_count_df_without_invalid_user['taskId'].isin(
        ['VIETNAMTRIP_WITH_QUESTIONUI'])
]
all_condition_vietnam_visit_web_page_count_df = pd.concat([
    vietnam_visit_web_page_count_df,
    q_vietnam_visit_web_page_count_df
])
anova_result = kruskal(
    data=all_condition_vietnam_visit_web_page_count_df, dv="visit_web_page_count", between='taskId')
display(anova_result)
if int(anova_result['p-unc'] < 0.05):
    mcomp_result = do_multiple_comparison_with_FDR_vietnam(
        all_condition_vietnam_visit_web_page_count_df, target_value='visit_web_page_count')
    display(mcomp_result)




### SERP閲覧時間

In [ ]:
# 大気汚染
airPollution_serp_time_duration_df = serp_time_duration_df_without_invalid_user[
    serp_time_duration_df_without_invalid_user['taskId'].isin(
        AIRPOLLUTION_UI_CONDITION)
]
q_airPollution_serp_time_duration_df = q_serp_time_duration_df_without_invalid_user[
    q_serp_time_duration_df_without_invalid_user['taskId'].isin(
        ['AIRPOLLUTION_WITH_QUESTIONUI'])
]
all_condition_airPollution_serp_time_duration_df = pd.concat(
    [airPollution_serp_time_duration_df, q_airPollution_serp_time_duration_df])
anova_result = kruskal(
    data=all_condition_airPollution_serp_time_duration_df,
    dv="time_on_serp",
    between='taskId'
)
display(anova_result)
if int(anova_result['p-unc'] < 0.05):
    mcomp_result = do_multiple_comparison_with_FDR_airPollution(
        all_condition_airPollution_serp_time_duration_df, target_value='time_on_serp')
    display(mcomp_result)

# ベトナム旅行
vietnam_serp_time_duration_df = serp_time_duration_df_without_invalid_user[
    serp_time_duration_df_without_invalid_user['taskId'].isin(
        VIETNAMTRIP_UI_CONDITION)
]
q_vietnam_serp_time_duration_df = q_serp_time_duration_df_without_invalid_user[
    q_serp_time_duration_df_without_invalid_user['taskId'].isin(
        ['VIETNAMTRIP_WITH_QUESTIONUI'])
]
all_condition_vietnam_serp_time_duration_df = pd.concat([
    vietnam_serp_time_duration_df,
    q_vietnam_serp_time_duration_df
])
anova_result = kruskal(
    data=all_condition_vietnam_serp_time_duration_df, dv="time_on_serp", between='taskId')
display(anova_result)
if int(anova_result['p-unc'] < 0.05):
    mcomp_result = do_multiple_comparison_with_FDR_vietnam(
        all_condition_vietnam_serp_time_duration_df, target_value='time_on_serp')
    display(mcomp_result)


### 最大クリック深度

In [ ]:
# 大気汚染
airPollution_max_ranking_df = max_ranking_df_without_invalid_user[
    max_ranking_df_without_invalid_user['taskId'].isin(
        AIRPOLLUTION_UI_CONDITION)
]
q_airPollution_max_ranking_df = q_max_ranking_df_without_invalid_user[
    q_max_ranking_df_without_invalid_user['taskId'].isin(
        ['AIRPOLLUTION_WITH_QUESTIONUI'])
]
all_condition_airPollution_max_ranking_df = pd.concat(
    [airPollution_max_ranking_df, q_airPollution_max_ranking_df])
anova_result = kruskal(
    data=all_condition_airPollution_max_ranking_df,
    dv="max_ranking",
    between='taskId'
)
display(anova_result)
if int(anova_result['p-unc'] < 0.05):
    mcomp_result = do_multiple_comparison_with_FDR_airPollution(
        all_condition_airPollution_max_ranking_df, target_value='max_ranking')
    display(mcomp_result)

# ベトナム旅行
vietnam_max_ranking_df = max_ranking_df_without_invalid_user[
    max_ranking_df_without_invalid_user['taskId'].isin(
        VIETNAMTRIP_UI_CONDITION)
]
q_vietnam_max_ranking_df = q_max_ranking_df_without_invalid_user[
    q_max_ranking_df_without_invalid_user['taskId'].isin(
        ['VIETNAMTRIP_WITH_QUESTIONUI'])
]
all_condition_vietnam_max_ranking_df = pd.concat([
    vietnam_max_ranking_df,
    q_vietnam_max_ranking_df
])
anova_result = kruskal(
    data=all_condition_vietnam_max_ranking_df, dv="max_ranking", between='taskId')
display(anova_result)
if int(anova_result['p-unc'] < 0.05):
    mcomp_result = do_multiple_comparison_with_FDR_vietnam(
        all_condition_vietnam_max_ranking_df, target_value='max_ranking')
    display(mcomp_result)


## 事後アンケート分析

In [ ]:
questionnaire_tpp_question_ui_csv_path = './data/再実験_事後アンケート_問いかけボットUI_ベトナム旅行（回答）.csv'
questionnaire_air_pollution_question_ui_csv_path = './data/再実験_事後アンケート_問いかけボットUI_大気汚染（回答）.csv'
questionnaire_vietnam_question_ui_csv_path = './data/再実験_事後アンケート_問いかけボットUI_ベトナム旅行（回答）.csv'
questionnaire_perspective_ui_csv_path = './data/意見形成タスク事後アンケート_観点一覧UI.csv'
questionnaire_non_ui_csv_path = './data/意見形成タスク事後アンケート_NonUI.csv'

questionnaire_tpp_question_ui_df = pd.read_csv(
    questionnaire_tpp_question_ui_csv_path)
questionnaire_air_pollution_question_ui_df = pd.read_csv(questionnaire_air_pollution_question_ui_csv_path)
questionnaire_vietnam_question_ui_df = pd.read_csv(questionnaire_vietnam_question_ui_csv_path)
questionnaire_perspective_ui_df = pd.read_csv(questionnaire_perspective_ui_csv_path)
questionnaire_non_ui_df = pd.read_csv(questionnaire_non_ui_csv_path)

questionnaire_tpp_question_ui_df.head()

# カラム名の置換
replace_columns_dict_for_question_ui = {
    'あなたは，タスクで与えられたテーマについて，幅広い観点でウェブ検索を行うことができたと思いますか？': 'various_perspective_search',
    'あなたは，タスクで与えられたテーマについて，ウェブ検索を行った結果，理解をより深めることができましたか？': 'deepen_understanding_theme',
    'あなたは，問いかけ文の提示は，タスクで与えられたテーマに関するあなた自身の理解度を確認するのに役立ちましたか？': 'check_understanding',
    'あなたは，ウェブ検索中に定期的に提示される問いかけ文は，それを必要とするタイミングで提示されたと思いますか？' : 'is_question_suggesting_appropriately',
    'あなたは，タスク中に提示された問いかけ文は，日本語文章として自然でしたか？': 'is_question_natural',
    'あなたは，タスク中に提示された問いかけ文について考えることは，与えられたテーマをより深く理解する上で役立ちましたか？': 'is_question_useful_for_deepen_understanding_theme',
    'あなたは，タスク中に提示された問いかけ文の内容について考えることは，意見をまとめる上で有用でしたか？': 'is_question_useful_for_forming_opinion',
    'あなたは，タスクであなたがまとめた意見は，説得力の高いものだと思いますか？': 'is_your_opinion_highly_persuasive',
    'あなたは，タスクであなたがまとめた意見に満足していますか?': 'are_you_satisfied_your_opinion',
    'あなたは，問いかけ提示機能をもつウェブ検索システムは使いやすいと思いますか？': 'is_system_easy_to_use',
    '今後ウェブ検索を使って何か意見をまとめることがある時，あなたは今回使用した問いかけ提示機能をもつウェブ検索システムを使ってみたいと思いますか？': 'do_you_use_the_system_again',
    'あなたが最後に卒業なさったあるいは在学中の教育機関は次のどれですか？（中退は卒業に含まれません）': 'last_educational_background',
    'この値は変更しないでください．\r\n値が変更された場合，報酬をお支払いできない場合があります．': 'crowdId'
}
replace_columns_dict_for_not_question_ui = {
    'あなたは，タスクで与えられたテーマについて，幅広い観点でウェブ検索を行うことができたと思いますか？': 'various_perspective_search',
    'あなたは，タスクで与えられたテーマについて，ウェブ検索を行った結果，理解をより深めることができましたか？': 'deepen_understanding_theme',
    'あなたは，タスクであなたがまとめた意見は，説得力の高いものだと思いますか？': 'is_your_opinion_highly_persuasive',
    'あなたは，タスクであなたがまとめた意見に満足していますか?': 'are_you_satisfied_your_opinion',
    'あなたが最後に卒業なさったあるいは在学中の教育機関は次のどれですか？（中退は卒業に含まれません）': 'last_educational_background',
    'この値は変更しないでください．\r\n値が変更された場合，報酬をお支払いできない場合があります．': 'crowdId'
}

# NOTE: あなたは，ウェブ検索中に定期的に提示された問いかけ文の提示頻度についてどう思いますか？については，個別で評価を行う．
no_use_columns_for_question_ui = [
    'あなたは，ウェブ検索中に定期的に提示された問いかけ文の提示頻度についてどう思いますか？',
    '6，問いかけ提示機能をもつウェブ検索システムについて感想があれば，記述してください．',
    '性別を教えて下さい',
    '年代を教えて下さい',
    '完了コード'
]
no_use_columns_for_not_question_ui = [
    '性別を教えて下さい',
    '年代を教えて下さい',
    '完了コード'
]

questionnaire_tpp_question_ui_df = questionnaire_tpp_question_ui_df.rename(
    columns=replace_columns_dict_for_question_ui
).drop(no_use_columns_for_question_ui, axis=1)
questionnaire_air_pollution_question_ui_df = questionnaire_air_pollution_question_ui_df.rename(
    columns=replace_columns_dict_for_question_ui
).drop(no_use_columns_for_question_ui, axis=1)
questionnaire_vietnam_question_ui_df = questionnaire_vietnam_question_ui_df.rename(
    columns=replace_columns_dict_for_question_ui
).drop(no_use_columns_for_question_ui, axis=1)
questionnaire_perspective_ui_df = questionnaire_perspective_ui_df.rename(
    columns=replace_columns_dict_for_not_question_ui
).drop(no_use_columns_for_not_question_ui, axis=1)
questionnaire_non_ui_df = questionnaire_non_ui_df.rename(
    columns=replace_columns_dict_for_not_question_ui
).drop(no_use_columns_for_not_question_ui, axis=1)


In [ ]:
# taskIdカラムを追加
questionnaire_tpp_question_ui_df['taskId'] = 'TPP_WITH_QUESTIONUI'
questionnaire_air_pollution_question_ui_df['taskId'] = 'AIRPOLLUTION_WITH_QUESTIONUI'
questionnaire_vietnam_question_ui_df['taskId'] = 'VIETNAMTRIP_WITH_QUESTIONUI'
questionnaire_tpp_question_ui_df.head()
not_question_ui_users = get_dataframe_from_db(
    'select * from users;', DB_CONFIG)
not_question_ui_users_df = get_dataframe_without_invalid_user(
    not_question_ui_users, valid_user_crowdIds)

# perspectiveUI群の事後アンケートdataframeにtaskIdを追加
# tpp
tpp_perspective_ui_users_df = not_question_ui_users_df.query(
    'taskId == "TPP_WITH_QUERYUI"')
tpp_perspective_ui_user_crowdIds = tpp_perspective_ui_users_df.crowdId.tolist()
questionnaire_tpp_perspective_ui_df = questionnaire_perspective_ui_df[questionnaire_perspective_ui_df['crowdId'].isin(
    tpp_perspective_ui_user_crowdIds)]
questionnaire_tpp_perspective_ui_df['taskId'] = 'TPP_WITH_QUERYUI'
# 大気汚染
air_pollution_perspective_ui_users_df = not_question_ui_users_df.query(
    'taskId == "AIRPOLLUTION_WITH_QUERYUI"')
air_pollution_perspective_ui_user_crowdIds = air_pollution_perspective_ui_users_df.crowdId.tolist()
questionnaire_air_pollution_perspective_ui_df = questionnaire_perspective_ui_df[questionnaire_perspective_ui_df['crowdId'].isin(
    air_pollution_perspective_ui_user_crowdIds)]
questionnaire_air_pollution_perspective_ui_df['taskId'] = 'AIRPOLLUTION_WITH_QUERYUI'
# ベトナム旅行
vietnam_perspective_ui_users_df = not_question_ui_users_df.query(
    'taskId == "VIETNAMTRIP_WITH_QUERYUI"')
vietnam_perspective_ui_user_crowdIds = vietnam_perspective_ui_users_df.crowdId.tolist()
questionnaire_vietnam_perspective_ui_df = questionnaire_perspective_ui_df[questionnaire_perspective_ui_df['crowdId'].isin(
    vietnam_perspective_ui_user_crowdIds)]
questionnaire_vietnam_perspective_ui_df['taskId'] = 'VIETNAMTRIP_WITH_QUERYUI'

# nonUI群の事後アンケートdataframeにtaskIdを追加
# tpp
tpp_non_ui_users_df = not_question_ui_users_df.query(
    'taskId == "TPP_WITH_NONUI"'
)
tpp_non_ui_user_crowdIds = tpp_non_ui_users_df.crowdId.tolist()
questionnaire_tpp_non_ui_df = questionnaire_non_ui_df[questionnaire_non_ui_df['crowdId'].isin(
    tpp_non_ui_user_crowdIds)]
questionnaire_tpp_non_ui_df['taskId'] = 'TPP_WITH_NONUI'
# 大気汚染
air_pollution_non_ui_users_df = not_question_ui_users_df.query(
    'taskId == "AIRPOLLUTION_WITH_NONUI"'
)
air_pollution_non_ui_user_crowdIds = air_pollution_non_ui_users_df.crowdId.tolist()
questionnaire_air_pollution_non_ui_df = questionnaire_non_ui_df[questionnaire_non_ui_df['crowdId'].isin(
    air_pollution_non_ui_user_crowdIds)]
questionnaire_air_pollution_non_ui_df['taskId'] = 'AIRPOLLUTION_WITH_NONUI'
# ベトナム旅行
vietnam_non_ui_users_df = not_question_ui_users_df.query(
    'taskId == "VIETNAMTRIP_WITH_NONUI"'
)
vietnam_non_ui_user_crowdIds = vietnam_non_ui_users_df.crowdId.tolist()
questionnaire_vietnam_non_ui_df = questionnaire_non_ui_df[questionnaire_non_ui_df['crowdId'].isin(
    vietnam_non_ui_user_crowdIds)]
questionnaire_vietnam_non_ui_df['taskId'] = 'VIETNAMTRIP_WITH_NONUI'


## すべての群で共通の評価項目について分析

In [ ]:
columns_used_in_only_question_ui = [
    'check_understanding',
    'is_question_suggesting_appropriately',
    'is_question_natural',
    'is_question_useful_for_deepen_understanding_theme',
    'is_question_useful_for_forming_opinion',
    'is_system_easy_to_use',
    'do_you_use_the_system_again',
]

# 問いかけ群のみで使用している評価項目を削除
# questionnaire_tpp_question_ui_df = questionnaire_tpp_question_ui_df.drop(
#     columns=columns_used_in_only_question_ui, axis=1
# )
# questionnaire_air_pollution_question_ui_df = questionnaire_air_pollution_question_ui_df.drop(
#     columns=columns_used_in_only_question_ui, axis=1
# )
# questionnaire_vietnam_question_ui_df = questionnaire_vietnam_question_ui_df.drop(
#     columns=columns_used_in_only_question_ui, axis=1
# )



### 事後アンケートの各項目について，中央値と標準偏差

In [ ]:
tpp_questionnaire_df = pd.concat([
    questionnaire_tpp_question_ui_df,
    questionnaire_tpp_perspective_ui_df,
    questionnaire_tpp_non_ui_df
]
)

tpp_questionnaire_df.median()
tpp_questionnaire_df.std()

air_pollution_questionnaire_df = pd.concat([
    questionnaire_air_pollution_question_ui_df,
    questionnaire_air_pollution_perspective_ui_df,
    questionnaire_air_pollution_non_ui_df
])

air_pollution_questionnaire_df.median()
air_pollution_questionnaire_df.std()

vietnam_questionnaire_df = pd.concat([
    questionnaire_vietnam_question_ui_df,
    questionnaire_vietnam_perspective_ui_df,
    questionnaire_vietnam_non_ui_df
])


vietnam_questionnaire_df.median()
vietnam_questionnaire_df.std()


In [ ]:
evaluation_items = (
    'various_perspective_search',
    'deepen_understanding_theme',
    'is_your_opinion_highly_persuasive',
    'are_you_satisfied_your_opinion',
    # 'last_educational_background', # 使わないかも
)

for evaluation_item in evaluation_items:
    print(evaluation_item)
    anova_result = kruskal(
        data=tpp_questionnaire_df, dv=evaluation_item, between='taskId')
    display(anova_result)
    if int(anova_result['p-unc'] < 0.05):
        mcomp_result = do_multiple_comparison_with_FDR_vietnam(
            tpp_questionnaire_df, target_value=evaluation_item)
        display(mcomp_result)

for evaluation_item in evaluation_items:
    print(evaluation_item)
    anova_result = kruskal(
        data=air_pollution_questionnaire_df, dv=evaluation_item, between='taskId')
    display(anova_result)
    if int(anova_result['p-unc'] < 0.05):
        mcomp_result = do_multiple_comparison_with_FDR_vietnam(
            air_pollution_questionnaire_df, target_value=evaluation_item)
        display(mcomp_result)


for evaluation_item in evaluation_items:
    print(evaluation_item)
    anova_result = kruskal(
        data=vietnam_questionnaire_df, dv=evaluation_item, between='taskId')
    display(anova_result)
    if int(anova_result['p-unc'] < 0.05):
        print(evaluation_item)
        mcomp_result = do_multiple_comparison_with_FDR_vietnam(
            vietnam_questionnaire_df, target_value=evaluation_item)
        display(mcomp_result)
